In [1]:
import polars as pl
import pandas as pd
import pyarrow as pa
import sqlite3

In [2]:
csv_path = "Resources/heart_attack_prediction_dataset.csv"
heart_df = pl.read_csv(csv_path)

In [3]:
heart_df.head()

Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
str,i64,str,i64,str,i64,i64,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,i64,f64,i64,i64,i64,str,str,str,i64
"""BMW7812""",67,"""Male""",208,"""158/88""",72,0,0,1,0,0,4.168189,"""Average""",0,0,9,6.615001,261404,31.251233,286,0,6,"""Argentina""","""South America""","""Southern Hemis…",0
"""CZE1114""",21,"""Male""",389,"""165/93""",98,1,1,1,1,1,1.813242,"""Unhealthy""",1,0,1,4.963459,285768,27.194973,235,1,7,"""Canada""","""North America""","""Northern Hemis…",0
"""BNI9906""",21,"""Female""",324,"""174/99""",72,1,0,0,0,0,2.078353,"""Healthy""",1,1,9,9.463426,235282,28.176571,587,4,4,"""France""","""Europe""","""Northern Hemis…",0
"""JLN3497""",84,"""Male""",383,"""163/100""",73,1,1,1,0,1,9.82813,"""Average""",1,0,9,7.648981,125640,36.464704,378,3,4,"""Canada""","""North America""","""Northern Hemis…",0
"""GFO8847""",66,"""Male""",318,"""91/88""",93,1,1,1,1,0,5.804299,"""Unhealthy""",1,0,6,1.514821,160555,21.809144,231,1,5,"""Thailand""","""Asia""","""Northern Hemis…",0


In [4]:
heart_df = heart_df.drop(['Obesity', 'Exercise Hours Per Week', 'Diet', 'Medication Use', 'Sedentary Hours Per Day', 'Hemisphere', 'Heart Attack Risk'])

In [5]:
heart_df = heart_df[['Patient ID', 'Age', 'Sex', 'Income', 'Country', 'Continent', 'BMI', 'Blood Pressure', 'Heart Rate', 'Cholesterol', 'Triglycerides', 'Diabetes', 'Family History', 'Previous Heart Problems', 'Alcohol Consumption', 'Smoking', 'Stress Level', 'Sleep Hours Per Day', 'Physical Activity Days Per Week']]

In [6]:
heart_df.head()

Patient ID,Age,Sex,Income,Country,Continent,BMI,Blood Pressure,Heart Rate,Cholesterol,Triglycerides,Diabetes,Family History,Previous Heart Problems,Alcohol Consumption,Smoking,Stress Level,Sleep Hours Per Day,Physical Activity Days Per Week
str,i64,str,i64,str,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""BMW7812""",67,"""Male""",261404,"""Argentina""","""South America""",31.251233,"""158/88""",72,208,286,0,0,0,0,1,9,6,0
"""CZE1114""",21,"""Male""",285768,"""Canada""","""North America""",27.194973,"""165/93""",98,389,235,1,1,1,1,1,1,7,1
"""BNI9906""",21,"""Female""",235282,"""France""","""Europe""",28.176571,"""174/99""",72,324,587,1,0,1,0,0,9,4,4
"""JLN3497""",84,"""Male""",125640,"""Canada""","""North America""",36.464704,"""163/100""",73,383,378,1,1,1,1,1,9,4,3
"""GFO8847""",66,"""Male""",160555,"""Thailand""","""Asia""",21.809144,"""91/88""",93,318,231,1,1,1,0,1,6,5,1


In [7]:
heart_df = heart_df.with_columns(pl.col('Diabetes').cast(pl.Boolean).alias('Diabetes'), 
           pl.col('Family History').cast(pl.Boolean).alias('Family History'),
           pl.col('Previous Heart Problems').cast(pl.Boolean).alias('Previous Heart Problems'),
           pl.col('Alcohol Consumption').cast(pl.Boolean).alias('Alcohol Consumption'),
           pl.col('Smoking').cast(pl.Boolean).alias('Smoking'),
           pl.col('BMI').cast(pl.Decimal(scale=2)).alias('BMI'),
           pl.col("Income").map_elements(lambda x: "${:,.0f}".format(x), return_dtype=pl.Object)
)

In [8]:
heart_df = heart_df.rename({'Patient ID': 'Patient_ID', 'Blood Pressure': 'Blood_Pressure', 'Heart Rate': 'Heart_Rate', 'Family History': 'Family_History',
                'Previous Heart Problems': 'Previous_Heart_Problems', 'Alcohol Consumption': 'Alcohol_Consumption', 'Stress Level': 'Stress_Level', 'Sleep Hours Per Day': 'Sleep_Hours',
                'Physical Activity Days Per Week': 'Active_Days'})

In [9]:
heart_df.head()

Patient_ID,Age,Sex,Income,Country,Continent,BMI,Blood_Pressure,Heart_Rate,Cholesterol,Triglycerides,Diabetes,Family_History,Previous_Heart_Problems,Alcohol_Consumption,Smoking,Stress_Level,Sleep_Hours,Active_Days
str,i64,str,object,str,str,f64,str,i64,i64,i64,bool,bool,bool,bool,bool,i64,i64,i64
"""BMW7812""",67,"""Male""","$261,404","""Argentina""","""South America""",31.25,"""158/88""",72,208,286,false,false,false,false,true,9,6,0
"""CZE1114""",21,"""Male""","$285,768","""Canada""","""North America""",27.19,"""165/93""",98,389,235,true,true,true,true,true,1,7,1
"""BNI9906""",21,"""Female""","$235,282","""France""","""Europe""",28.17,"""174/99""",72,324,587,true,false,true,false,false,9,4,4
"""JLN3497""",84,"""Male""","$125,640","""Canada""","""North America""",36.46,"""163/100""",73,383,378,true,true,true,true,true,9,4,3
"""GFO8847""",66,"""Male""","$160,555","""Thailand""","""Asia""",21.8,"""91/88""",93,318,231,true,true,true,false,true,6,5,1


In [10]:
#Create DataFrames for SQL tables
Patient_Info = heart_df[['Patient_ID', 'Age', 'Sex', 'Income', 'Country', 'Continent']]
Health_Information = heart_df[['Patient_ID', 'Cholesterol', 'Blood_Pressure', 'Heart_Rate', 'BMI', 'Triglycerides']]
Risk_Factors = heart_df[['Patient_ID', 'Smoking', 'Alcohol_Consumption', 'Family_History', 'Diabetes', 'Stress_Level', 'Previous_Heart_Problems']]
Lifestyle_Factors = heart_df[['Patient_ID', 'Active_Days', 'Sleep_Hours', 'Income', 'Smoking', 'Alcohol_Consumption']]

In [14]:
Patient_Info1 = heart_df.select(pl.col('Patient_ID'))


In [15]:
pandas = Patient_Info1.to_pandas()

In [16]:
pandas

,Patient_ID
0,BMW7812
1,CZE1114
2,BNI9906
3,JLN3497
4,GFO8847
...,...
8758,MSV9918
8759,QSV6764
8760,XKA5925
8761,EPE6801


In [13]:
Patient_Info.write_csv("test.csv")

ComputeError: csv writer does not support object dtype

In [11]:
Patient_df = Patient_Info.to_pandas()
Health_df = Health_Information.to_pandas()
Risk_df = Risk_Factors.to_pandas()
Lifestyle_df = Lifestyle_Factors.to_pandas()

PanicException: not implemented

In [ ]:
#Patient_df = Patient_Info.to_dict()

In [12]:
Patient_Info.to_dict()

PanicException: Object("object", Some(object-registry)) not impl

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('Heart_Attack_Risk_Dataset.db')

In [ ]:
# Specify the SQL table name to write to
patient_table = 'Patient_Info'
health_table = 'Health_Information'
risk_table = 'Risk_Factors'
lifestyle_table = 'Lifestyle_Factors'

In [ ]:
# Write the Polars DataFrame to the SQL table
Patient_Info.to_sql(patient_table, conn, index=False, if_exists='replace')
Health_Information.to_sql(health_table, conn, index=False, if_exists='replace')
Risk_Factors.to_sql(risk_table, conn, index=False, if_exists='replace')
Lifestyle_Factors.to_sql(lifestyle_table, conn, index=False, if_exists='replace')

AttributeError: 'DataFrame' object has no attribute 'write_sql_table'

In [ ]:
conn.close()